In [1]:
import os
import sys
import glob
sys.path.append('..')
sys.path.append('../scripts')
import xlrd
import pandas as pd
from pandas import DataFrame, Series

from ghs import hazard_classes
import ghs_jp
from ghs_jp import jp_haz_keywords, jp_haz_classes

In [2]:
chms = ghs_jp.extract()

In [3]:
df1 = DataFrame(chms).transpose()
df1.tail()

,CML_INPUT_ID,casrns,classifications,jp_ID,name
998-40-3,998-40-3,[998-40-3],"{'flamm_liq': {'date_classified': 'FY2009', 'h...",21A3656,Tributylphosphine
999-61-1,999-61-1,[999-61-1],"{'flamm_liq': {'date_classified': 'FY2006', 'h...",631,2-hydroxypropyl acrylate
999-81-5,999-81-5,[999-81-5],"{'flamm_liq': {'date_classified': 'FY2006', 'h...",1174,"Ethanaminium, 2-chloro-N,N,N-trimethyl-"
999-97-3,999-97-3,[999-97-3],"{'flamm_liq': {'date_classified': 'FY2012', 'h...",24A6122,Hexamethyldisilazane
99Mixtone},99Mixtone},[99Mixtone}],"{'flamm_liq': {'date_classified': 'FY2007', 'h...",99,"Mixture of 1,3,5-tris(3-aminomethylphenyl)-1,3..."


In [4]:
# Make a hazard-specific classification list for all chemicals in dataset.
# for h in sorted(hazard_keywords.keys()):
h = 'ozone'
haz_list = []
for c in chms.keys():
    if h not in chms[c]['classifications']:
        continue
    d = {k: chms[c][k] for k in chms[c].keys() \
         if k != 'classifications'}
    d['casrns'] = ', '.join(d['casrns'])
    d.update(chms[c]['classifications'][h])
    d['classification'] = jp_haz_classes[h] + ' - ' \
                          + d['classification']
    haz_list.append(d)

In [5]:
df2 = DataFrame(haz_list, columns=ghs_jp.xl_cols).set_index('CML_INPUT_ID')
df2.head()

,casrns,name,classification,hazard_statement,symbol,signal_word,rationale,country_code,jp_ID,date_classified,date_imported,source_filename
CML_INPUT_ID,,,,,,,,,,,,
95-83-0,95-83-0,4-chloro-o-phenylenediamine,Hazardous to the ozone layer - Classification ...,,,,,jp,708,FY2010,Sun Sep 20 16:33:06 2015,h22_mhlw_rev_e.xls
51-21-8,51-21-8,5-Fluorouracil,Hazardous to the ozone layer - Classification ...,,,,,jp,22A4093,FY2010,Sun Sep 20 16:33:04 2015,h22_mhlw_new_e.xls
634-66-2,634-66-2,"1,2,3,4-Tetrachlorobenzene",Hazardous to the ozone layer - Classification ...,,,,,jp,24A6117,FY2012,Sun Sep 20 16:33:05 2015,h24_mhlw_new_e.xls
111-71-7,111-71-7,Heptanal,Hazardous to the ozone layer - Classification ...,,,,,jp,23A5021,FY2011,Sun Sep 20 16:33:05 2015,h23_mhlw_new_e.xls
110-25-8,110-25-8,Oleoylsarcosine,Hazardous to the ozone layer - Classification ...,,,,,jp,25A0091,FY2013,Sun Sep 20 16:33:05 2015,h25_mhlw_new_e.xls


In [6]:
df2.classification.value_counts()

Hazardous to the ozone layer - Classification not possible    858
Hazardous to the ozone layer - Category 1                       3
dtype: int64

In [7]:
f = os.path.join(ghs_jp.JP_DATA_NEW_PATH, 'h21_mhlw_new_e.xls')
print(f)
book = xlrd.open_workbook(f)
sh = book.sheet_by_name('21A3673')
print(sh.row_values(45, 3, 8))

/Users/akokai/code/chemex/data/ghs/jp/new/h21_mhlw_new_e.xls
['Classification not possible', ' - ', ' - ', ' - ', 'No data available.']


In [8]:
nrows = []

new_search = os.path.join(ghs_jp.JP_DATA_NEW_PATH, '*.xl*')
new_files = glob.glob(new_search)

for f in new_files:
    book = xlrd.open_workbook(f)
    for i in range(1, book.nsheets): # Ignore the first sheet.
        sh = book.sheet_by_index(i)
        nrows.append(sh.nrows)

In [9]:
pd.Series(nrows).value_counts()

46    1998
47     873
dtype: int64